In [23]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Reading the Data
def read_dataset():
    df = pd.read_csv('sonar.csv')
    #print (len(df.column))
    X = df[df.columns[0:60]].values
    y = df[df.columns[60]]
    #Encode the dependent variable
    encoder = LabelEncoder()
    encoder.fit(y)
    y = encoder.transform(y)
    print(X.shape)
    return (X,y)
# Define the encoder function.
def one_hot_encoder(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels),labels]=1
    return one_hot_encode
# read the dataset
X,Y = read_dataset()

# Shuffle the dataset to mix up the rows
X,Y = shuffle(X,Y ,random_state=1)
# Convert the dataset into train and test part
train_x ,test_x ,train_y ,test_y = train_test_split(X,Y ,test_size =0.20,random_state =0)
#Inspect th shape of the training and testing
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)

# Define the important parameters and variable tot work with the tensors
learning_rate = 0.3
training_epochs = 1000
cost_history = np.empty(shape=[1],dtype=float)
n_dim = X.shape[1]
print("n_dim",n_dim)
n_class=2
model_path = "E:\testpychram"
#Model the number of hidden layers and the number of neurons for each layer
n_hidden_1 =60
n_hidden_2 = 60
n_hidden_3 = 60
n_hidden_4  =60
x = tf.placeholder(tf.float32,[None,n_dim])
W = tf.Variable(tf.zeros([n_dim ,n_class]))
b = tf.Variable(tf.zeros([n_class]))
y_ = tf.placeholder(tf.float32,[None ,n_class])

#Define the model
def multilayer_perceptron(x,weights ,biases):

    #Hidden layer with RELU activation
    layer_1= tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    # Hidden layer with sigmoid activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    # Hidden layer with sigmoid activation
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    layer_3 = tf.nn.sigmoid(layer_3)
    # Hidden layer with RELU activation
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4'])
    layer_4 = tf.nn.relu(layer_4)
    #Output layer witj linear activation
    out_layer = tf.matmul(layer_4,weights['out'])+biases['out']
    return out_layer
# Define the weights and biases for each layer
weights = {
    'h1':tf.Variable(tf.random.truncated_normal([n_dim ,n_hidden_1])),
    'h2':tf.Variable(tf.random.truncated_normal([n_hidden_1 ,n_hidden_2])),
     'h3':tf.Variable(tf.random.truncated_normal([n_hidden_2 ,n_hidden_3])),
     'h4':tf.Variable(tf.random.truncated_normal([n_hidden_3 ,n_hidden_4])),
     'out':tf.Variable(tf.random.truncated_normal([n_hidden_4 ,n_class]))
}
biases={
    'b1':tf.Variable(tf.random.truncated_normal([n_hidden_1])),
    'b2':tf.Variable(tf.random.truncated_normal([n_hidden_2])),
    'b3':tf.Variable(tf.random.truncated_normal([n_hidden_3])),
    'b4':tf.Variable(tf.random.truncated_normal([n_hidden_4])),
    'out':tf.Variable(tf.random.truncated_normal([n_class]))
}

# Initialize all the varibales
init = tf.global_variables_initializer()
saver = tf.train.Saver()

#Call yours model defined
y_ = multilayer_perceptron(x,weights,biases)

# Define the cost function and optimizer
cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y,labels =y_))
training_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)
sess = tf.Session()
sess.run(init)

# Calculate the cost and the accuracy for each epoch
mse_history = []
accuracy_history = []
for epoch in range(training_epochs):
    sess.run(training_step ,feed_dict={x:train_x,y_:train_y})
    cost = sess.run(cost_function,feed_dict={x:train_x,y_:train_y})
    cost_history =np.append(cost_history,cost)
    correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    #print("Accuracy:",(sess.run(accuracy,feed_dict={x:test_x,y_:test_y
    pred_y = sess.run(y,feed_dict={x:test_x})
    mse = tf.reduce_mean(tf.square(pred_y,test_y))
    mse_ = sess.run(mse)
    mse_history.append(mse_)
    accuracy = (sess.run(accuracy,feed_dict={x:train_x,y_:train_y}))
    accuracy_history.append(accuracy)
    print('epoch:',epoch, '-','cost:' ,cost, "MSE:",mse_,"Train Accuracy:",accuracy)
save_path = saver.save(sess,model_path)
print("Model saved in file :%s"%save_path)

#plot mse and accuracy graph
plt.plot(mse_history,'r')
plt.show()
#print the final accuracy
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print("Test Accuracy:",(sess.run(accuracy,feed_dict={x:test_x,y:test_y})))
#print the final mean square error
pred_y = sess.run(y,feed_dict={x:test_x})
mse = tf.reduce_mean(tf.square(pred_y-test_y))
print("MSE:%.4f"%sess.run(mse))
print(pd.__version__)

(207, 60)
(165, 60)
(165,)
(42, 60)
n_dim 60


ValueError: Cannot feed value of shape (165,) for Tensor 'add_14:0', which has shape '(?, 2)'